In [5]:
#필요 라이브러리 호출
import os
import base64
import requests
import pandas as pd
from PIL import Image
from io import BytesIO
from langchain import hub
import dataframe_image as dfi
from selenium import webdriver
from selenium.webdriver.common.by import By
from langchain_openai import OpenAIEmbeddings
from langchain_core.messages import HumanMessage
from langchain_community.vectorstores import FAISS
from langchain_community.chat_models import ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

# Chat GPT API 호출키
os.environ['OPENAI_API_KEY'] = 'My-API-KEY'
os.environ['USER_AGENT'] = 'myagent'

#출처: https://github.com/teddylee777/langchain-kr/blob/main/12-RAG/10-Multi_modal_RAG-GPT-4o.ipynb

In [18]:
def table_2_img_save(get_tables,save_dic):
    for i in range(len(get_tables)):
        table=get_tables[i]

        path=save_dic+"표 "+str(i+1)+'.jpeg'
        styled_df = table.style.set_properties(**{
            'text-align' : 'left',
            'white-space' : 'pre-wrap',
            'width' : 'auto'
        })
        styled_df.set_table_styles([{
            'selector': 'th',
            'props': [('text-align', 'center')]  # 헤더 텍스트 정렬
        }])
        dfi.export(styled_df, path, max_cols = -1, max_rows = -1)

In [7]:
##이미지 저장 및 요약 코드
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

def image_summarize(img_base64, prompt):
    chat = ChatOpenAI(model="gpt-4o", max_tokens=2048)
    msg = chat.invoke(
        [HumanMessage(content=[{"type": "text", "text": prompt},
                               {"type": "image_url",
                                "image_url": {"url": f"data:image/jpeg;base64,{img_base64}"},
                                },])])
    return msg.content
prompt = "Retrieval 하기 위하여 이미지를 요약해야해.원본 이미지 없이 임베딩에 이용하여 바로 적용될 것이니 상세히 요약해줘."

def img_save(get_url,save_dic):
    for i in range(len(get_url)):
        url=get_url[i][0].get_attribute('src')
        path=save_dic+'그림 '+str(i+1)+".jpeg"
        img_data = requests.get(url)
        imgg1=Image.open(BytesIO(img_data.content)).convert('RGB')
        imgg1.save(path,'JPEG')

def img_summary(path):
    bs64_img=encode_image(path)
    img_summary=image_summarize(bs64_img, prompt)
    return img_summary

In [73]:
#웹 크롤링
url="https://fconline.nexon.com/news/notice/view?n4ArticleSN=4893"

driver=webdriver.Chrome()

driver.get(url)

# get main text & image
# 피파모바일: p
# 테일즈위버: MsoNormal
text=driver.find_elements(By.TAG_NAME,"p")

# 요약 주제 추출
# 피파모바일 subject
# 테일즈위버 class=view_header -> tag=h3
#topic=driver.find_element(By.CLASS_NAME, "view_header").find_element(By.TAG_NAME, "h3")
topic=driver.find_element(By.CLASS_NAME,"subject")

#이미지 저장
img_save_dic='C:/Users/wjdgh/OneDrive/바탕 화면/과제 A/A-1/image_save/'
try:
    img=[]
    for i in driver.find_elements(By.CLASS_NAME, "MsoNormal"):
        if i.find_elements(By.TAG_NAME,'img'):
            img.append(i.find_elements(By.TAG_NAME,'img'))
    img_save(img,img_save_dic)
except: print('이미지 안 됨')

#테이블 이미지 변환 후 저장
try:
    table=pd.read_html(url)
    table_2_img_save(table,img_save_dic)
except: print('테이블 안 됨')

In [64]:
#텍스트 부분 정리
text_only=[]
try:
    table_check=driver.find_element(By.TAG_NAME,'table')
    for i in range(len(text)):
        if text[i] not in table_check.find_elements(By.TAG_NAME,'p') and text[i].get_attribute('class')=="MsoNormal":
            text_only.append(text[i].text)
except: ""

#url 창 닫기
driver.close()

text_only1 = list(map(lambda x : x.strip().replace(u"\xa0",u""), text_only))
text_only2 = list(map(lambda x : x.replace(u"\n",u""), text_only1))
text_only3=list(text for text in text_only2 if len(text)>0)

In [66]:
#이미지 요약 내용 텍스트 추가

img_load_dic = 'C:/Users/wjdgh/OneDrive/바탕 화면/과제 A/A-1/image_save/'
try:
    file_list = os.listdir(img_load_dic)
    for file in file_list:
        text_only3.append(img_summary(img_load_dic+file))
except: print()

In [67]:
texts=" ".join(text_only3)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=70,
    length_function=len,
    is_separator_regex=False,
)
texts1 = text_splitter.create_documents([texts])

In [69]:
#요약 내용 임베딩 및 VectorDB 추가
embeddings = OpenAIEmbeddings()

# FAISS 데이터베이스 생성 및 retriever 추가
db = FAISS.from_documents(texts1, embeddings)
retirevers=db.as_retriever()

In [70]:
#문서 요약 RAG
openai=ChatOpenAI(model="gpt-3.5-turbo",streaming=True,callbacks=[StreamingStdOutCallbackHandler()],temperature=0)
retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")
combine_docs_chain = create_stuff_documents_chain(openai, retrieval_qa_chat_prompt,)
retrieval_chain = create_retrieval_chain(retirevers, combine_docs_chain,)

In [71]:
#결과 도출
answer=retrieval_chain.invoke({"input": topic.text+"에 대하여 3줄로 요약해줘."})

# [7/25(목) 갱신] 7월 유료화 상품 판매 안내
# https://fconline.nexon.com/news/notice/view?n4ArticleSN=4893
# 7월 25일(목)에 판매 중인 유료화 상품 안내입니다.
# 상품에 포함된 확률형 아이템의 확률 정보는 전체 확률 정보를 통해 확인 가능합니다.
# 일부 유료화 상품에는 구매 제한이 적용되어 있습니다.

# "판매되고 있는 상품에 대해서 설명해줘"
# 현재 7월 25일(목)에 판매 중인 유료화 상품들이 있습니다. 
# 이 상품들은 확률형 아이템의 확률 정보를 확인할 수 있으며, 구매 제한이 적용된 상품도 있습니다. 
# 추가로 7월에 추가될 유료화 상품에 대한 안내도 이후에 공지될 예정입니다. 
# 자세한 상품명과 판매 위치는 FC 온라인 상품 목록 표에서 확인할 수 있습니다.

# EURO 결승 기념: 사전 접속 이벤트 안내
# https://fconline.nexon.com/news/notice/view?n4ArticleSN=4911
#"유로 결승전 사전 이벤트의 기간, 참여 방법, 보상에 대하여 3줄로 요약해줘."
#유로 결승전 사전 이벤트는 2024년 7월 14일 하루 동안 진행됩니다.
#FC온라인(PC)과 FC온라인 M(모바일)에 접속하여 보상을 획득할 수 있으며, 각 게임 최초 접속 시 보상이 지급됩니다.
#보상으로는 EURO FINAL 상자를 포함한 다양한 아이템이 제공됩니다.

#"유로 결승전 사전 이벤트에 대하여 3줄로 요약해줘."
# 7/14(일)에는 유로 2024 결승전을 기념하여 사전 접속 이벤트가 진행됩니다.
# PC나 모바일로 접속하여 보상을 받을 수 있으며, 최초 접속 시에만 보상이 지급됩니다.
# 보상에는 EURO FINAL 상자, EU24 Kit 상자, 스페셜 코치 팩, 전송 수수료 30% 할인 쿠폰 등이 포함됩니다.

# EURO 결승 기념: 사전 접속 이벤트 안내
# https://fconline.nexon.com/news/notice/view?n4ArticleSN=4911
# EURO 2024 결승전을 기념하여 7월 14일 하루 동안 PC와 모바일에서 사전 접속 이벤트가 진행됩니다.
# 이벤트 기간 동안 최초 접속 시 'EURO FINAL 상자' 등 다양한 보상을 받을 수 있습니다.
# 보상은 게임 내 보관함으로 자동 지급되며, 모바일 접속 시 최소 1분 이상 유지해야 합니다.


# 게임플레이 버그 제보 게시판 이용 안내
# https://fconline.nexon.com/news/notice/view?n4ArticleSN=4929
# GM리버풀에서는 7/11(목) 정기점검을 통해 '게임 플레이 버그 제보 게시판'을 추가했습니다.
# 7/22(월)에는 게시물 작성 양식과 분류 카테고리가 업데이트되었는데, 이를 통해 구단주님들께서 더 편리하게 제보를 할 수 있도록 도와드립니다.
# 유튜브 영상 등록 방법과 함께 적극적인 제보를 부탁드리며, 버그 현상 파악에 도움이 될 것입니다


# 7/18(목) 전력분석실 업데이트 안내
# https://fconline.nexon.com/news/notice/view?n4ArticleSN=4923
# 7/18(목) 정기점검을 통해 전력분석실의 플레이 스타일이 업데이트되었습니다.
# '빈틈 공략'과 '측면 공략' 플레이 스타일이 다시 추가되었고, '좌측면 선호'와 '철퇴 축구' 플레이 스타일이 삭제되었습니다.
# 전력분석실은 [프리롬 > 훈련센터 > 전력분석실]에서 이용할 수 있습니다.


# 스페셜 출석체크 이벤트
# https://tales.nexon.com/News/Notice/129681
# 2020년 3월 19일부터 6월 17일까지 진행되는 '스페셜 출석체크 이벤트'가 있습니다.
# 매일 접속하여 공격력 및 방어력 상승 버프와 다양한 아이템을 획득할 수 있습니다.
# 이벤트 기간 중 200레벨 이상 캐릭터로 참여 가능하며, 출석체크를 완료하지 못한 경우 SEED를 사용하여 보충할 수 있습니다.

# 스페셜 출석 이벤트의 기간 참여방법 보상내역을 3줄로 요약해줘.
# 2020년 3월 19일부터 6월 17일까지 진행되는 스페셜 출석체크 이벤트.
# 게임에 접속하여 30분 후 보상을 받을 캐릭터를 클릭하거나 이벤트 창에서 보상을 받을 수 있음.
# 보상으로는 공격력 및 방어력 +20% 버프, 아이템, 초대장 등이 제공됨.

# 출석 보상을 설명해줘
# 출석체크 이벤트에서는 매일 게임에 접속하여 출석을 완료하면 다양한 보상을 받을 수 있습니다.
# 출석 보상으로는 스페셜 스킬, 던전 초대장, 아바타, 변신망토, 간판, 아이디 툴팁, 펫 툴팁 염색약, 이모티콘 등이 제공됩니다.
# 또한, 공격력 및 방어력 +20% 버프를 받아 1시간 동안 적용되며, 출석 보상 아바타 아이템을 착용하면 화려한 아바타 5세트 효과를 얻을 수 있습니다.
# 이벤트 종료 후에도 보상 획득 및 받지 못한 보상의 출석 보충이 가능하니 참고하시기 바랍니다.

7월 25일(목)에 판매 중인 유료화 상품 안내입니다. 상품에 포함된 확률형 아이템의 확률 정보는 전체 확률 정보[L]를 통해 확인 가능합니다. 일부 유료화 상품에는 구매 제한이 적용되어 있습니다.